# Atividade 03

Nesse script, será desenvolvido:
* Uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas.

* A função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade.

*  Implementar a função de estimativa de sobrelevação da pluma utilizando os métodos de Davidson-Bryant, Holland e Briggs. Deve ser considerado o efeito Tip-Downwash

*  Função do modelo gaussiano

* Realizar simulações com o script criado, utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491.


In [6]:
import numpy as np
import pandas as pd

# Definindo a quantidade de dados
num_values = 20

# Gerando vetor de datas para 5 dias no verão e 5 dias no inverno
summer_dates = pd.date_range(start='2023-12-06', periods=10, freq='D')  # Dias de verão no Brasil 
winter_dates = pd.date_range(start='2023-06-21', periods=10, freq='D')  # Dias de inverno no Brasil
dates = summer_dates.append(winter_dates)

# Gerando vetor de cobertura de nuvens (valores entre 0 e 8)
cloud_cover = np.random.randint(0, 9, size=num_values)

# Gerando vetor de radiação solar usando o método de Monte Carlo com seno e cosseno
# Assumindo valores entre 0 e 1000 W/m^2
random_factors = np.random.rand(num_values)  # fatores aleatórios entre 0 e 1
solar_radiation = 500 + 500 * np.sin(2 * np.pi * random_factors) * np.cos(2 * np.pi * random_factors)

# Gerando vetor de velocidade do vento usando o método de Monte Carlo com seno e cosseno
# Assumindo valores entre 0 e 15 m/s
wind_speed = 7.5 + 7.5 * np.sin(2 * np.pi * random_factors) * np.cos(2 * np.pi * random_factors)

# Criando DataFrame para organizar os dados
data = pd.DataFrame({
    'Date': dates,
    'Cloud Cover': cloud_cover,
    'Solar Radiation (W/m^2)': solar_radiation,
    'Wind Speed (m/s)': wind_speed
})
print(data)


         Date  Cloud Cover  Solar Radiation (W/m^2)  Wind Speed (m/s)
0  2023-12-06            8               499.789341          7.496840
1  2023-12-07            8               737.455857         11.061838
2  2023-12-08            1               687.658194         10.314873
3  2023-12-09            0               671.545626         10.073184
4  2023-12-10            8               666.663303          9.999950
5  2023-12-11            6               256.909289          3.853639
6  2023-12-12            4               749.692075         11.245381
7  2023-12-13            0               451.283413          6.769251
8  2023-12-14            5               486.145442          7.292182
9  2023-12-15            6               724.264747         10.863971
10 2023-06-21            6               250.099665          3.751495
11 2023-06-22            6               740.568314         11.108525
12 2023-06-23            3               743.990255         11.159854
13 2023-06-24       

In [7]:
import pandas as pd

# Função para classificar a estabilidade de acordo com Pasquil
def classify_stability(row):
    wind_speed = row['Wind Speed (m/s)']
    cloud_cover = row['Cloud Cover']
    solar_radiation = row['Solar Radiation (W/m^2)']
    
    # Classificação baseada em cobertura de nuvens e velocidade do vento
    if cloud_cover <= 3:
        if wind_speed < 2:
            if solar_radiation > 600:
                return 'A'
            elif solar_radiation > 300:
                return 'B'
            else:
                return 'C'
        elif wind_speed < 3:
            if solar_radiation > 600:
                return 'A'
            elif solar_radiation > 300:
                return 'B'
            else:
                return 'C'
        elif wind_speed < 5:
            if solar_radiation > 600:
                return 'B'
            elif solar_radiation > 300:
                return 'C'
            else:
                return 'D'
        else:
            if solar_radiation > 600:
                return 'C'
            elif solar_radiation > 300:
                return 'D'
            else:
                return 'D'
    elif cloud_cover <= 6:
        if wind_speed < 3:
            return 'D'
        elif wind_speed < 5:
            return 'D'
        else:
            return 'D'
    else:
        if wind_speed < 2:
            return 'E'
        elif wind_speed < 3:
            return 'E'
        elif wind_speed < 5:
            return 'D'
        else:
            return 'D'

# Carregar os dados fictícios gerados anteriormente
data = pd.DataFrame({
    'Date': pd.date_range(start='2024-01-01', periods=20, freq='D'),
    'Cloud Cover': np.random.randint(0, 9, size=20),
    'Solar Radiation (W/m^2)': 500 + 500 * np.sin(2 * np.pi * np.random.rand(20)) * np.cos(2 * np.pi * np.random.rand(20)),
    'Wind Speed (m/s)': 7.5 + 7.5 * np.sin(2 * np.pi * np.random.rand(20)) * np.cos(2 * np.pi * np.random.rand(20))
})

# Aplicando a função de classificação
data['Stability Class'] = data.apply(classify_stability, axis=1)

print(data)


         Date  Cloud Cover  Solar Radiation (W/m^2)  Wind Speed (m/s)  \
0  2024-01-01            7                75.984205          7.559534   
1  2024-01-02            0               860.349142         11.001721   
2  2024-01-03            2               549.407513          1.462601   
3  2024-01-04            6               629.049613         11.853067   
4  2024-01-05            7               192.387702          9.689186   
5  2024-01-06            2               309.820127          9.861909   
6  2024-01-07            0               928.563554         14.760062   
7  2024-01-08            6               369.162564          7.366146   
8  2024-01-09            5               330.573869          9.211401   
9  2024-01-10            4               682.732506          4.042298   
10 2024-01-11            7               616.573946          9.216108   
11 2024-01-12            0               966.666367          9.716623   
12 2024-01-13            1               281.769200

## 2) Implementação da função sigma YZ para estimar os coeficientes de dispersão

In [39]:
def sigmaYZ(x, classe, urbOrRural):
    if urbOrRural == 'urbano':
        if classe == 'A' or classe == 'B':
            sigmaY = 0.32 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.24 * x * (1 + 0.001 * x) ** 0.5
        elif classe == 'C':
            sigmaY = 0.22 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'D':
            sigmaY = 0.16 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.14 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe == 'E' or classe == 'F':
            sigmaY = 0.11 * x * (1 + 0.0004 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0015 * x) ** (-0.5)
        else:
            raise ValueError('Classe de estabilidade errada')
    elif urbOrRural == 'rural':
        if classe == 'A':
            sigmaY = 0.22 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.20 * x
        elif classe == 'B':
            sigmaY = 0.16 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.12 * x
        elif classe == 'C':
            sigmaY = 0.11 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.08 * x * (1 + 0.0002 * x) ** (-0.5)
        elif classe == 'D':
            sigmaY = 0.08 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.06 * x * (1 + 0.0003 * x) ** (-0.5)
        elif classe == 'E':
            sigmaY = 0.06 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.03 * x * (1 + 0.0003 * x) ** (-1)
        elif classe == 'F':
            sigmaY = 0.04 * x * (1 + 0.0001 * x) ** (-0.5)
            sigmaZ = 0.016 * x * (1 + 0.0003 * x) ** (-1)
        else:
            raise ValueError('Classe de estabilidade errada')
    else:
        raise ValueError('Tipo de ambiente deve ser "urbano" ou "rural"')

    # Adicionando declarações de impressão para depuração
    print(f"Classe: {classe}, Ambiente: {urbOrRural}, x: {x}")
    print(f"sigmaY: {sigmaY}, sigmaZ: {sigmaZ}")

    return sigmaY, sigmaZ

# Exemplo de chamada da função
x = 1000
classe = 'C'
urbOrRural = 'urbano'
sigmaY, sigmaZ = sigmaYZ(x, classe, urbOrRural)
print(f"Resultado: sigmaY = {sigmaY}, sigmaZ = {sigmaZ}")


Classe: C, Ambiente: urbano, x: 1000
sigmaY: 185.93393604027364, sigmaZ: 200.0
Resultado: sigmaY = 185.93393604027364, sigmaZ = 200.0


# 3) Implementação do método de Briggs para cálculo da sobrelevação da pluma. 

In [28]:
def briggs_plume_rise(wind_speed, stack_height, exit_velocity, stack_diameter, delta_temp, temp_ambient, stability_class):
    """
    Calcula a sobrelevação da pluma usando a fórmula de Briggs considerando classes de estabilidade.
    
    Parâmetros:
    - wind_speed: velocidade do vento (m/s)
    - stack_height: altura da chaminé (m)
    - exit_velocity: velocidade de saída dos gases (m/s)
    - stack_diameter: diâmetro da chaminé (m)
    - delta_temp: diferença de temperatura entre os gases de saída e o ar ambiente (K)
    - temp_ambient: temperatura do ar ambiente (K)
    - stability_class: classe de estabilidade de Pasquil (C ou D)
    
    Retorna:
    - plume_rise: sobrelevação da pluma (m)
    """
    
    g = 9.81  # aceleração gravitacional (m/s^2)
    buoyancy_flux = (g * stack_diameter**2 * exit_velocity * delta_temp) / (4 * temp_ambient)  # fluxo de flutuabilidade
    
    if stability_class == 'C':
        plume_rise = (2.6 * buoyancy_flux**0.333 * stack_height**0.667) / (wind_speed + 0.5 * exit_velocity)
    elif stability_class == 'D':
        plume_rise = (1.6 * buoyancy_flux**0.333 * stack_height**0.667) / (wind_speed + 0.5 * exit_velocity)
    else:
        raise ValueError('Classe de estabilidade inválida. Use "C" ou "D".')
    
    return plume_rise

# Parâmetros fixos
stack_height = 45 # m
exit_velocity = 10  # m/s
stack_diameter = 3  # m
delta_temp = 150  # K
temp_ambient = 298  # K

# Aplicando a função ao DataFrame para calcular a pluma
data['Plume Rise'] = data.apply(lambda row: briggs_plume_rise(
    row['Wind Speed (m/s)'],
    stack_height,
    exit_velocity,
    stack_diameter,
    delta_temp,
    temp_ambient,
    stability_class
), axis=1)

## 4) implementar o modelo gaussiano

In [29]:
# H ALTURA CHAMINÉ + SOBRELEVAÇÃO 
#Z ALTURA DO NARIZ
#Y DIREÇÃO PERPENDICULAR AO EIXO DA PLUMA 
#VELOCIDADE DO VENTO NO DIA
#QS TAXA DE MEISSÃO DO EMPRENDIEMNTO.

Observações de parametros utilizados: Como o estudo de poluição atmosférica deve ser simulado no pior cenário, aqui será considerado o menor valor de sobrelevação da pluma, de modo a simular o momento em que, mesmo com a pluma mais baixa, a altura da chaminé é suficente para dispersar poluentes.
Como H =  Maior altura de sobrelevação da pluma + altura da chaminé, foi aferida a altura de 45M para a chaminé do empreendimento da tupy, em Joinville, através da ferramenta goolge earth. 


In [44]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def processar_dados():
    print("Processando dados...")

def exibir_imagem():
    # Carregar a imagem
    img = mpimg.imread('caminho/para/sua/imagem.jpg')

    # Exibir a imagem
    plt.imshow(img)
    plt.axis('off')  # Esconder os eixos
    plt.show()

def continuar_processo():
    print("Continuação do processo...")

# Execução do código
processar_dados()
exibir_imagem()
continuar_processo()


SyntaxError: invalid syntax (1688137497.py, line 5)

In [25]:
# Encontrando o maior valor da pluma
h = data[['Plume Rise C', 'Plume Rise D']].min().min()
print(f"O menor valor da pluma é: {h:.2f} m")

O menor valor da pluma é: 5.28 m


In [40]:
# Criando uma função do modelo gaussiano
H = h + 45 # menor valor de elevação da pluma + altura da chaminé
qs= 91270.27 #g/s #PM-10





import numpy as np
def modeloGaussiano(qs,sigmaY,sigmaZ,u,y,z,H):
    termo1 = qs/(2*np.pi*sigmaY*sigmaZ*u)
    termo2 = np.exp((-y**2)/(2*sigmaY**2))
    termo3 = np.exp((-(z-H)**2)/(2*sigmaZ**2)) + np.exp((-(z+H)**2)/(2*sigmaZ**2))
    conc = termo1*termo2*termo3
    return conc
print (H) 
print (qs)
print (sigmaY)
print (sigmaZ)
print (u)
print (y)
print (z)
print(conc)

50.281674055379035
91270.27
185.93393604027364
200.0


NameError: name 'u' is not defined